# References

## Finding critical points using NNs
The idea is basically to focus only on the transition times between wake/sleep, training a custom model (Recurrent + ResNet cpts) to do it

model = MultiResidualBiGRU(input_size=10,hidden_size=64,out_size=2,n_layers=5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3,weight_decay = 0)

Bidirectional GRU takes inputs in the forward and backwards directions, and we combine with residual layers

https://ieeexplore.ieee.org/document/9836276

MultiResidualBiGRU combines 5 of these

https://www.kaggle.com/code/werus23/sleep-critical-point-infer


## Random Forest model starter
Trained on a subset of the full series: 35/277, excluding noisy datasets

https://www.kaggle.com/code/carlmcbrideellis/zzzs-random-forest-model-starter

## Estimating sleep parameters using an accelerometer
Using sleep-detection algorithm based off of meeting a rolling median threshold combined with block length to create extra features

https://www.nature.com/articles/s41598-018-31266-z

## XGBoost Regressor
Create a feature measuring how far off the measurement model is from accurate

https://analyticsindiamag.com/how-to-use-xgboost-for-time-series-analysis/



In [ ]:
import os
from datetime import datetime
import random
import math

import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from scipy.interpolate import interp1d
from math import pi, sqrt, exp
import sklearn
import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
from sklearn.metrics import average_precision_score
from sklearn.ensemble import RandomForestClassifier
from itertools import groupby
import pickle

import pyarrow as pa
from pyarrow.parquet import ParquetFile

# GPU Configuration
import ctypes
torch.set_num_interop_threads(4)
torch.set_num_threads(4)

# Device Selection
device = 'cuda' if torch.cuda.is_available() else 'cpu'


## Config/Data Loading

In [ ]:
class PATHS:
    MAIN_DIR = "/kaggle/input/child-mind-institute-detect-sleep-states/"
    # CSV FILES : 
    SUBMISSION = MAIN_DIR + "sample_submission.csv"
    TRAIN_EVENTS = MAIN_DIR + "train_events.csv"
    # PARQUET FILES:
    TRAIN_SERIES = MAIN_DIR + "train_series.parquet"
    TEST_SERIES = MAIN_DIR + "test_series.parquet"

class data_reader:
    def __init__(self):
        super().__init__()
        # MAPPING FOR DATA LOADING :
        self.names_mapping = {
            "submission" : {"path" : PATHS.SUBMISSION, "is_parquet" : False, "has_timestamp" : False}, 
            "train_events" : {"path" : PATHS.TRAIN_EVENTS, "is_parquet" : False, "has_timestamp" : True},
            "train_series" : {"path" : PATHS.TRAIN_SERIES, "is_parquet" : True, "has_timestamp" : True},
            "test_series" : {"path" : PATHS.TEST_SERIES, "is_parquet" : True, "has_timestamp" : True}
        }
        self.valid_names = ["submission", "train_events", "train_series", "test_series"]
    
    def verify(self, data_name):
        "function for data name verification"
        if data_name not in self.valid_names:
            print("PLEASE ENTER A VALID DATASET NAME, VALID NAMES ARE : ", valid_names)
        return
    
    def cleaning(self, data):
        "cleaning function : drop na values"
        before_cleaning = len(data)
        print("Number of missing timestamps : ", len(data[data["timestamp"].isna()]))
        data = data.dropna(subset=["timestamp"])
        after_cleaning = len(data)
        print("Percentage of removed rows : {:.1f}%".format(100 * (before_cleaning - after_cleaning) / before_cleaning) )
        return data
    
    @staticmethod
    def reduce_memory_usage(data):
        "iterate through all the columns of a dataframe and modify the data type to reduce memory usage."
        start_mem = data.memory_usage().sum() / 1024**2
        print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
        for col in data.columns:
            col_type = data[col].dtype    
            if col_type != object:
                c_min = data[col].min()
                c_max = data[col].max()
                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        data[col] = data[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        data[col] = data[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        data[col] = data[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        data[col] = data[col].astype(np.int64)  
                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        data[col] = data[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        data[col] = data[col].astype(np.float32)
                    else:
                        data[col] = data[col].astype(np.float64)
            else:
                data[col] = data[col].astype('category')

        end_mem = data.memory_usage().sum() / 1024**2
        print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
        print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
        return data
    
    def load_data(self, data_name):
        "function for data loading"
        self.verify(data_name)
        data_props = self.names_mapping[data_name]
        if data_props["is_parquet"]:
            data = pd.read_parquet(data_props["path"])
        else:
            data = pd.read_csv(data_props["path"])
                
        gc.collect()
        if data_props["has_timestamp"]:
            print('cleaning')
            data = self.cleaning(data)
            gc.collect()
        data = self.reduce_memory_usage(data)
        return data

In [ ]:
werus_reader = data_reader()
test_series = werus_reader.load_data(data_name="test_series")
ids = test_series.series_id.unique()
gc.collect()

## Feature Engineering

In [ ]:
def rolling_window_algo(df_group):
    
    # Steps 3-5
    df_group['rolling_median'] = df_group['anglezdiff'].rolling(window=60).median()
    # Steps 6-7
    df_group['day_id'] = (df_group['timestamp'] - pd.Timedelta(hours=12)).dt.date
    thresholds = df_group.groupby('day_id')['rolling_median'].quantile(0.1) * 15
    df_group['threshold'] = df_group['day_id'].map(thresholds)
    below_threshold_series = (df_group['rolling_median'] < df_group['threshold']).astype('int')
    if below_threshold_series.empty or below_threshold_series.sum() == 0:
        df_group['rolling_algo_awake'] = 0
        df_group.drop(columns=['day_id', 'threshold', 'rolling_median'], inplace=True)
        gc.collect()
        print(df_group)
        return df_group
    df_group['below_threshold'] = below_threshold_series
    block_diff_series = below_threshold_series.diff()
    df_group['block_diff'] = block_diff_series
    df_group['block_start'] = (block_diff_series == 1).astype('int')
    df_group['block_end'] = (block_diff_series == -1).astype('int')
    if below_threshold_series.iloc[0] == 1:
        df_group.at[0, 'block_start'] = 1
    if below_threshold_series.iloc[-1] == 1:
        df_group.at[-1, 'block_end'] = 1
    block_start_times = df_group.loc[df_group['block_start'] == 1, 'timestamp'].values
    block_end_times = df_group.loc[df_group['block_end'] == 1, 'timestamp'].values
    block_durations = block_end_times - block_start_times
    valid_block_mask = block_durations > np.timedelta64(30, 'm')
    df_group['valid_block'] = 0
    df_group.loc[df_group['block_start'] == 1, 'valid_block'] = valid_block_mask.astype(int)
    df_group.loc[df_group['block_end'] == 1, 'valid_block'] = valid_block_mask.astype(int)
    # Step 8
    gap_durations = block_start_times[1:] - block_end_times[:-1]
    valid_gap_mask = gap_durations < np.timedelta64(60, 'm')
    gap_start_indices = df_group[df_group['block_end'] == 1].index[:-1][valid_gap_mask]
    gap_end_indices = df_group[df_group['block_start'] == 1].index[1:][valid_gap_mask]
    df_group.loc[gap_start_indices, 'valid_block'] = 1
    df_group.loc[gap_end_indices, 'valid_block'] = 1
    # Step 9
    cum_valid_block = df_group['valid_block'].cumsum()
    sleep_period_length = df_group.groupby(['day_id', cum_valid_block])['valid_block'].sum()
    longest_block_index = sleep_period_length.idxmax()
    df_group['main_sleep_period'] = 0
    df_group.loc[(df_group['day_id'] == longest_block_index[0]) & (cum_valid_block == longest_block_index[1]), 'main_sleep_period'] = 1
    df_group['rolling_algo_awake'] = (~(df_group['valid_block'] | df_group['main_sleep_period'])).astype(int).fillna(method="ffill")
    columns_to_drop = ['day_id', 'threshold', 'below_threshold', 'block_start', 'block_end', 'block_diff', 'valid_block', 'main_sleep_period', 'rolling_median']
    df_group.drop(columns=columns_to_drop, inplace=True)
    gc.collect()
    print(df_group)
    return df_group

In [ ]:
def make_features(df):
    # parse the timestamp and create an "hour" feature
    df['series_id'] = df['series_id'].astype('category')
    df['timestamp'] = pd.to_datetime(df['timestamp']).apply(lambda t: t.tz_localize(None))
    df["hour"] = df["timestamp"].dt.hour
    df['minute'] = df['timestamp'].dt.minute
    # Use .fillna(value = 0) or .interpolate(method='linear')?
    df["anglezdiff"] = df["anglez"].diff().abs().astype(np.float32)

    df.sort_values(['timestamp'], inplace=True)

    # Rolling window algo

    df = df.groupby('series_id').apply(rolling_window_algo).reset_index(drop=True)

    new_columns = []
        
    # periods in seconds        
    periods = [60, 360, 720] 
    for col in ['enmo', 'anglez', 'anglezdiff']:
        
        for n in periods:
            
            
            rol_args = {'window': int(n/5), 'min_periods':10, 'center':True}
            
            for agg in ['median', 'mean', 'max', 'min', 'var']:
                new_col_name = f'{col}_{agg}_{n}'
                new_col = df[col].rolling(**rol_args).agg(agg).astype(np.float32)
                new_col.name = new_col_name
                new_columns.append(new_col)

    df = pd.concat([df] + new_columns, axis=1)

    df.drop(columns=['timestamp'], inplace=True)
    df.dropna(inplace=True)

    return df

## Model Initialization

In [ ]:
class ResidualBiGRU(nn.Module):
    def __init__(self, hidden_size, n_layers=1, bidir=True):
        super(ResidualBiGRU, self).__init__()

        self.hidden_size = hidden_size
        self.n_layers = n_layers

        self.gru = nn.GRU(
            hidden_size,
            hidden_size,
            n_layers,
            batch_first=True,
            bidirectional=bidir,
        )
        dir_factor = 2 if bidir else 1
        self.fc1 = nn.Linear(
            hidden_size * dir_factor, hidden_size * dir_factor * 2
        )
        self.ln1 = nn.LayerNorm(hidden_size * dir_factor * 2)
        self.fc2 = nn.Linear(hidden_size * dir_factor * 2, hidden_size)
        self.ln2 = nn.LayerNorm(hidden_size)

    def forward(self, x, h=None):
        res, new_h = self.gru(x, h)
        # res.shape = (batch_size, sequence_size, 2*hidden_size)

        res = self.fc1(res)
        res = self.ln1(res)
        res = nn.functional.relu(res)

        res = self.fc2(res)
        res = self.ln2(res)
        res = nn.functional.relu(res)

        # skip connection
        res = res + x

        return res, new_h

class MultiResidualBiGRU(nn.Module):
    def __init__(self, input_size, hidden_size, out_size, n_layers, bidir=True):
        super(MultiResidualBiGRU, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.out_size = out_size
        self.n_layers = n_layers

        self.fc_in = nn.Linear(input_size, hidden_size)
        self.ln = nn.LayerNorm(hidden_size)
        self.res_bigrus = nn.ModuleList(
            [
                ResidualBiGRU(hidden_size, n_layers=1, bidir=bidir)
                for _ in range(n_layers)
            ]
        )
        self.fc_out = nn.Linear(hidden_size, out_size)

    def forward(self, x, h=None):
        # if we are at the beginning of a sequence (no hidden state)
        if h is None:
            # (re)initialize the hidden state
            h = [None for _ in range(self.n_layers)]

        x = self.fc_in(x)
        x = self.ln(x)
        x = nn.functional.relu(x)

        new_h = []
        for i, res_bigru in enumerate(self.res_bigrus):
            x, new_hi = res_bigru(x, h[i])
            new_h.append(new_hi)

        x = self.fc_out(x)
#         x = F.normalize(x,dim=0)
        return x, new_h  # log probabilities + hidden states

In [ ]:
class SleepDataset(Dataset):
    def __init__(
        self,
        series_ids,
        series,
    ):
        series_ids = series_ids
        series = series.reset_index()
        self.data = []
        
        for viz_id in tqdm(series_ids):
            self.data.append(series.loc[(series.series_id==viz_id)].copy().reset_index())
            
    def downsample_seq_generate_features(self,feat, downsample_factor):
        
        if len(feat)%12!=0:
            feat = np.concatenate([feat,np.zeros(12-((len(feat))%12))+feat[-1]])
        feat = np.reshape(feat, (-1,12))
        feat_mean = np.mean(feat,1)
        feat_std = np.std(feat,1)
        feat_median = np.median(feat,1)
        feat_max = np.max(feat,1)
        feat_min = np.min(feat,1)

        return np.dstack([feat_mean,feat_std,feat_median,feat_max,feat_min])[0]
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        X = self.data[index][['anglez','enmo']].values.astype(np.float32)
        X = np.concatenate([self.downsample_seq_generate_features(X[:,i],12) for i in range(X.shape[1])],-1)
        X = torch.from_numpy(X)
        return X
werus_test_ds = SleepDataset(test_series.series_id.unique(),test_series)
# del werus_test_series
# gc.collect()

In [ ]:
max_chunk_size = 24*60*100
min_interval = 30
model = MultiResidualBiGRU(input_size=10,hidden_size=64,out_size=2,n_layers=5).to(device).eval()
model.load_state_dict(torch.load(f'/kaggle/input/werus-rf-ensemble/werus_model.pth',map_location=device))

In [ ]:
# import classifier models
classifier = pickle.load(open('/kaggle/input/xgb-pickle-test/xgb_test.pkl', "rb"))

## Inference

In [ ]:
# Here Werus downsamples to save computation, so we get the dataframe of 12-step data, and will use it to interpolate the predictions back to 1-step data
# werus_submission = pd.DataFrame()
werus_preds = pd.DataFrame()
for i in range(len(werus_test_ds)):
    X = werus_test_ds[i].half()
    
    seq_len = X.shape[0]
    h = None
    pred = torch.zeros((len(X),2)).half()
    for j in range(0, seq_len, max_chunk_size):
        y_pred, h = model(X[j: j + max_chunk_size].float(), h)
        h = [hi.detach() for hi in h]
        pred[j : j + max_chunk_size] = y_pred.detach()
        del y_pred;gc.collect()
    del h,X;gc.collect()
    pred = pred.numpy()
    
    series_id = ids[i]
    
    days = len(pred)/(17280/12)
    scores0,scores1 = np.zeros(len(pred),dtype=np.float16),np.zeros(len(pred),dtype=np.float16)
    for index in range(len(pred)):
        if pred[index,0]==max(pred[max(0,index-min_interval):index+min_interval,0]):
            scores0[index] = max(pred[max(0,index-min_interval):index+min_interval,0])
        if pred[index,1]==max(pred[max(0,index-min_interval):index+min_interval,1]):
            scores1[index] = max(pred[max(0,index-min_interval):index+min_interval,1])

    steps = werus_test_ds.data[i][['step']].iloc[np.clip(range(len(pred))*12, 0, len(werus_test_ds.data[i])-1)].astype(np.int32)
    temp_df = pd.DataFrame()
    temp_df['series_id'] = [series_id] * len(steps)
    temp_df['step'] = steps['step'].values
    temp_df['werus_score_0'] = scores0
    temp_df['werus_score_1'] = scores1
    # candidates_onset = np.argsort(scores0)[-max(1,round(days)):]
    # candidates_wakeup = np.argsort(scores1)[-max(1,round(days)):]
    
    # onset = werus_test_ds.data[i][['step']].iloc[np.clip(candidates_onset*12,0,len(werus_test_ds.data[i])-1)].astype(np.int32)
    # onset['event'] = 'onset'
    # onset['series_id'] = series_id
    # onset['score']= scores0[candidates_onset]
    # wakeup = werus_test_ds.data[i][['step']].iloc[np.clip(candidates_wakeup*12,0,len(werus_test_ds.data[i])-1)].astype(np.int32)
    # wakeup['event'] = 'wakeup'
    # wakeup['series_id'] = series_id
    # wakeup['score']= scores1[candidates_wakeup]
    werus_preds = pd.concat([werus_preds,temp_df],axis=0)
    # werus_submission = pd.concat([werus_submission,onset,wakeup],axis=0)
    # del onset,wakeup,candidates_onset,candidates_wakeup,scores0,scores1,pred,series_id,
    del scores0,scores1,pred,series_id,temp_df
    gc.collect()
del werus_test_ds
gc.collect()

In [ ]:
# werus_submission = werus_submission.sort_values(['series_id','step']).reset_index(drop=True)
# werus_submission['row_id'] = werus_submission.index.astype(int)
# werus_submission['score'] = werus_submission['score'].fillna(werus_submission['score'].mean())
# werus_submission = werus_submission[['row_id','series_id','step','event','score']]

In [ ]:
def get_series_counts(df):
    return df.groupby('series_id').size()

def distribute_series_ids(series_counts):
    sorted_series_ids = series_counts.sort_values(ascending=False).index.tolist()
    if len(series_counts) > 10:
        distributed_ids = [[] for _ in range(10)]
        total_rows = [0] * 10
    else:
        distributed_ids = [[] for _ in range(len(series_counts))]
        total_rows = [0] * len(series_counts)
    for series_id in sorted_series_ids:
        idx = np.argmin(total_rows)
        distributed_ids[idx].append(series_id)
        total_rows[idx] += series_counts[series_id]
    return distributed_ids

In [ ]:
gb_result_df = pd.DataFrame(columns=["series_id", "step", "not_awake", "awake"])

# Distribute series_ids into 10 subsets
series_counts = get_series_counts(test_series)
distributed_ids = distribute_series_ids(series_counts)

# Iterate through each subset of series_ids, process data and run inference
for i, series_id_list in enumerate(distributed_ids):
    # Load data for the current subset of series_ids
    chunk_df = test_series[test_series['series_id'].isin(series_id_list)]
    
    # Perform feature engineering
    chunk_df = make_features(chunk_df)
     # Run inference
    chunk_df["not_awake"] = classifier.predict_proba(chunk_df.loc[:, chunk_df.columns != "series_id"])[:,0]
    chunk_df["awake"] = classifier.predict_proba(chunk_df.loc[:, ~chunk_df.columns.isin(["series_id", "not_awake"])])[:,1]
    
    # Append results to gb_result_df
    result_chunk = chunk_df[["series_id", "step", "not_awake", "awake"]]
    gb_result_df = pd.concat([gb_result_df, result_chunk], ignore_index=True)
    del chunk_df
    gc.collect()

In [ ]:
test = gb_result_df
del gb_result_df
gc.collect()

In [ ]:
# test  = make_features(test_series)

# del test_series
# gc.collect()

# test["not_awake"] = classifier.predict_proba(test.loc[:, test.colums != "series_id"])[:,0]
# test["awake"]     = classifier.predict_proba(test.loc[:, test.colums != "series_id"])[:,1]

In [ ]:
# test = test[['series_id','step','not_awake','awake', 'rolling_algo_awake', 'anglezdiff', 'enmo', 'enmo_mean_60', 'enmo_var_60', 'minute', 'hour']]
# gc.collect()

## Ensembling

In [ ]:
# Make a final ensemble df with all the predictions
ensemble_df = test.merge(werus_preds, on=['series_id', 'step'], how='left')
del test,werus_preds
gc.collect()
# Interpolate missing values in "score0" and "score1" columns
ensemble_df['werus_score_0'] = ensemble_df['werus_score_0'].interpolate(method='linear')
ensemble_df['werus_score_1'] = ensemble_df['werus_score_1'].interpolate(method='linear')

In [ ]:
ensemble_df['not_awake'] = ensemble_df['not_awake'].interpolate(method='linear')
ensemble_df['awake'] = ensemble_classifier.predict_proba(ensemble_df.loc[:, ensemble_df.columns != "series_id"])[:,1]

In [ ]:
# smoothing the predictions
smoothing_length = 2*230
test["score"]  = test["awake"].rolling(smoothing_length,center=True).mean().fillna(method="bfill").fillna(method="ffill")
test["smooth"] = test["not_awake"].rolling(smoothing_length,center=True).mean().fillna(method="bfill").fillna(method="ffill")
# re-binarize
test["smooth"] = test["smooth"].round()

# https://stackoverflow.com/questions/73777727/how-to-mark-start-end-of-a-series-of-non-null-and-non-0-values-in-a-column-of-a
def get_event(df):
    lstCV = zip(df.series_id, df.smooth)
    lstPOI = []
    for (c, v), g in groupby(lstCV, lambda cv: 
                            (cv[0], cv[1]!=0 and not pd.isnull(cv[1]))):
        llg = sum(1 for item in g)
        if v is False: 
            lstPOI.extend([0]*llg)
        else: 
            lstPOI.extend(['onset']+(llg-2)*[0]+['wakeup'] if llg > 1 else [0])
    return lstPOI

test["event"] = get_event(test)

In [ ]:
rf_submission = test.loc[test["event"] != 0][["series_id","step","event","score"]].copy().reset_index(drop=True).reset_index(names="row_id")
del test   
gc.collect()
